notebook to test and find best hyperparameters for various models

In [6]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import RandomizedSearchCV as RSCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

In [7]:
import os

pid = os.getpid()
print("PID: %i" % pid)

n_cpu = os.cpu_count()   # Number of CPUs assigned to this process
print("Number of CPUs in the system:", n_cpu)

# we won't use all the available cpu's for this script 
n_jobs = n_cpu - 2 # The number of tasks to run in parallel

# Control which CPUs are made available for this script
cpu_arg = ''.join([str(ci) + ',' for ci in list(range(n_jobs))])[:-1]
cmd = 'taskset -cp %s %i' % (cpu_arg, pid)
print("executing command '%s' ..." % cmd)
os.system(cmd)

PID: 26064
Number of CPUs in the system: 16
executing command 'taskset -cp 0,1,2,3,4,5,6,7,8,9,10,11,12,13 26064' ...


1

In [8]:
# path to cleaned data file
trainFilePath = 'data/data_train.pkl'

# Read already prepared and saved train datasets
with open(trainFilePath, 'rb') as f:
    data_train = pickle.load(f)

# Create dummy variables for categorical data
data_train = pd.get_dummies(data_train, columns=['clinical_stage', 'biopsy_gleason_gg', 'pathological_gleason_gg',
                                'pathologic_stage', 'lni', 'surgical_margin_status', 'persistent_psa',
                                'TRYSgrupes', 'PLNDO1'])

In [23]:
"""
Explodes the provided "df" dataset based on provided survival column "time" and
clips the data to be in a range [min_time; max_time] (). A new discrete survival column
will be created with name set as variable "time_discrete". "cum_event" boolean determines
if cumulative event column will be created or no.

clip(lower, upper) function will help us create a new discrete survival time column. 
If we specify lower=1 and upper=200, patients who experienced event earlier than 200th 
month will only have records till their event, on other side, if a patient survived past 
200th month, we will clip this information and will only keep information about him til 200th month.
Another example, if we specify lower=140 and upper=200, and if the person experienced event 
at 100th month, we will create records for him till 140th (lower boundary) month.
"""
def explode_data(df,max_time,time,target_column,min_time=1,
                 time_discrete='survival_time_discrete',cum_event=False):

    target_column_discrete = target_column + '_discrete'

    # We create a new time column and clip the data by provided min and max survival times
    df[time_discrete] = df[time].clip(min_time,max_time).apply(range)

    # Exploding the dataset with the created range value in new time column
    data_exploded = df.explode(time_discrete)
    data_exploded.reset_index(drop=True, inplace=True)

    # New column starts at 0, we'll increase each value by 1
    data_exploded[time_discrete] = pd.to_numeric(data_exploded[time_discrete]) + 1

    # New event column, which will indicate the last event date
    data_exploded[target_column_discrete] = (data_exploded[time_discrete] >= data_exploded[time]) * pd.to_numeric(data_exploded[target_column])
    
    if cum_event == True:
        target_column_cumulative = target_column + '_cumulative'

        # Create new event column with duplicated event values from discrete column
        data_exploded[target_column_cumulative] = data_exploded[target_column_discrete]
        
        # For cumulative events, after end_time we will have NA values, we'll replace those with event indicator
        after_survival_time = data_exploded[time_discrete] > data_exploded[time]
        data_exploded.loc[after_survival_time, target_column_discrete] = -1
        data_exploded[target_column_discrete] = data_exploded[target_column_discrete].replace(-1,np.NaN)
        data_exploded.loc[(after_survival_time & (data_exploded[target_column]==0)), target_column_cumulative] = -1
        data_exploded[target_column_cumulative] = data_exploded[target_column_cumulative].replace(-1,np.NaN)

    return data_exploded


"""
Given an exploded dataset with instant mortality probabilities "event_probability_column"
and "id_column" for grouping (optional), cumulative hazard column will be calculated
"""
def cumulative_hazard(df, event_probability_column, id_column):
    data_copy = df.copy()
    if id_column is not None:
        data_copy = data_copy[ [id_column, event_probability_column] ]
    else:
        data_copy = data_copy[ [event_probability_column] ]
    data_copy['negative_log_prob'] = np.log( 1 - data_copy[event_probability_column] )
    if id_column is not None:
        data_copy['cumulative_hazard'] = 1 - np.exp(data_copy.groupby(id_column)['negative_log_prob'].transform(pd.Series.cumsum))
    else:
        data_copy['cumulative_hazard'] = 1 - np.exp(data_copy['negative_log_prob'].cumsum())
    return data_copy['cumulative_hazard']



def add_predict_probabilities_optimized(df_exploded, target_column, model):
    """
    Given exploded datase. Adds predictend instant mortality probabilities as well as cumulative ones.
    """

    df_exploded_copy = df_exploded.copy()
    df_exploded_copy.drop(['patient_id', target_column + '_cumulative'], axis=1, inplace=True)
    
    # probabilities
    y_pred = model.predict_proba(df_exploded_copy)[:,1]
    df_exploded['mortality_instant_prob'] = y_pred

    # Cumulative hazard for each patient
    df_exploded['cumulative_hazard'] = cumulative_hazard(df_exploded,'mortality_instant_prob','patient_id')
    
    return df_exploded


def optimize_model_stratifiedkfold(target_column, max_time, space, df, model_type, max_evals):
    data_train_copy = df.copy().reset_index(drop=True)

    # mts and bcr have different survival months columns
    match target_column:
        case 'mts':
            time = 'survival_months_mts'
        case 'bcr':
            time = 'survival_months_bcr'
        case _:
            time = 'survival_months'

    target_column_discrete = target_column + '_discrete'

    # List of columns names which will be dropped from feature set before fitting the model
    target_columns = ['cancer_specific_mortality', 'death_from_other_causes', 'bcr', 'mts']

    def objective(space):

        # Create StratifiedKFold object.
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

        auc_cv = []

        for train_index, val_index in skf.split(data_train_copy, data_train_copy.loc[:,target_column]):
            
            # get train and validation sets
            train = data_train_copy.loc[train_index,:]
            val = data_train_copy.loc[val_index,:]
            
            # explode train set
            train_exploded = explode_data(train.copy(), min_time=1, max_time=max_time, time=time, target_column=target_column)

            # explode validation set with cumulative events
            val_exploded_cumulative = explode_data(val.copy(), max_time=max_time, min_time=max_time, cum_event=True, time=time, target_column=target_column)

            # define model
            match model_type:
                case 'RF':
                    model = RandomForestClassifier(criterion = space['criterion'], max_depth = int(space['max_depth']),
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = int(space['n_estimators']), 
                                 )
                case 'XGBoost':
                    model = XGBClassifier(objective="binary:logistic", random_state=0, booster='gbtree', eval_metric='auc',
                        colsample_bytree=space['colsample_bytree'], gamma=space['gamma'], learning_rate=space['learning_rate'],
                        max_depth=int(space['max_depth']), n_estimators=int(space['n_estimators']), subsample=space['subsample'],
                        min_child_weight=space['min_child_weight'], scale_pos_weight=space['scale_pos_weight'], 
                        reg_lambda=space['reg_lamda'], reg_alpha=space['reg_alpha'])
                case 'NN':
                    model = MLPClassifier(hidden_layer_sizes=space['hidden_layer_sizes'],
                        alpha=space['alpha'], tol=1e-4, 
                        random_state=1, activation=space['activation'], 
                        learning_rate=space['learning_rate'],
                        max_iter=space['max_iter'],
                        solver=space['solver']
                       )
                case _:
                    pass
            
            # columns to drop before fitting
            x_columns_to_drop = [target_column_discrete, 'survival_months', 'survival_months_bcr', 'survival_months_mts', 'patient_id']
            x_columns_to_drop.extend(target_columns)

            # train X and y
            X_train = train_exploded.drop(x_columns_to_drop, axis=1)
            y_train = train_exploded[target_column_discrete]

            # fit model
            model.fit(X_train, y_train)

            # columns to drop before making predictions
            x_columns_to_drop = [target_column_discrete, 'survival_months', 'survival_months_bcr', 'survival_months_mts']
            x_columns_to_drop.extend(target_columns)

            # add predicted probabilities
            df_val_exploded_pred = add_predict_probabilities_optimized(val_exploded_cumulative.drop(x_columns_to_drop,axis=1), target_column, model)

            # calculate auc
            # AUC for each cumulative slice
            # Months at which we'll check the AUC's
            months = list(range(6, max_time, 6))

            val_auc_stats = []
            for month in months:
                # --- Validation data ---
                # Selecting a subset of data based on the months
                select = (df_val_exploded_pred['survival_time_discrete'] == month) & pd.notna(df_val_exploded_pred[target_column+'_cumulative'])
                sub_dat = df_val_exploded_pred[select]

                # If in the sliced data there's a event, calculate AUC metric,
                # otherwise assign NaN value
                if sub_dat[target_column+'_cumulative'].max() == 1:
                    fpr, tpr, thresholds = roc_curve(sub_dat[target_column+'_cumulative'], sub_dat['cumulative_hazard'])
                    auc_stat = auc(fpr, tpr)
                else:
                    auc_stat = float('NaN')
                val_auc_stats.append(auc_stat)

            auc_mean = np.nanmean(val_auc_stats)

            auc_cv.append(auc_mean)

        auc_cv_mean = np.nanmean(auc_cv)

        # We aim to maximize auc, therefore we return it as a negative value
        return {'loss': -auc_cv_mean, 'status': STATUS_OK }
    
    trials = Trials()
    best = fmin(fn = objective,
                space = space,
                algo = tpe.suggest,
                max_evals = max_evals,
                trials = trials)
    
    print(best)
    
    return trials, best

## Random Forest

### Bayesian Optimization

#### cancer specific mortality

In [5]:
space_model = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.uniform('max_depth',5,20),
        'max_features': hp.choice('max_features', ['sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.uniform('n_estimators', 100, 500)
    }

trials, best = optimize_model_stratifiedkfold(target_column='cancer_specific_mortality',
                                              max_time=216,
                                              space=space_model,
                                              df = data_train.copy(),
                                              model_type='RF',
                                              max_evals=80)

100%|██████████| 80/80 [28:19<00:00, 21.25s/trial, best loss: -0.8930259526960415]
{'criterion': 1, 'max_depth': 13.574069212358445, 'max_features': 0, 'min_samples_leaf': 0.03168091798222092, 'min_samples_split': 0.4308128762421261, 'n_estimators': 308.1562762776964}


#### death from other causes

In [ ]:
space_model = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.uniform('max_depth',5,20),
        'max_features': hp.choice('max_features', ['sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.uniform('n_estimators', 100, 500)
    }

trials, best = optimize_model_stratifiedkfold(target_column='death_from_other_causes',
                                              max_time=216,
                                              space=space_model,
                                              df = data_train.copy(),
                                              model_type='RF',
                                              max_evals=80)

#### mts

In [ ]:
space_model = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.uniform('max_depth',5,20),
        'max_features': hp.choice('max_features', ['sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.uniform('n_estimators', 100, 500)
    }

trials, best = optimize_model_stratifiedkfold(target_column='mts',
                                              max_time=216,
                                              space=space_model,
                                              df = data_train.copy(),
                                              model_type='RF',
                                              max_evals=80)

#### bcr

In [ ]:
space_model = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.uniform('max_depth',5,20),
        'max_features': hp.choice('max_features', ['sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.uniform('n_estimators', 100, 500)
    }

trials, best = optimize_model_stratifiedkfold(target_column='bcr',
                                              max_time=216,
                                              space=space_model,
                                              df = data_train.copy(),
                                              model_type='RF',
                                              max_evals=80)

### XGBoost

#### cancer specific mortality

In [ ]:
space_model = {
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'gamma': hp.quniform('gamma', 0, 1, 0.05),
    'learning_rate': hp.quniform('learning_rate', 0, 0.2, 0.025),
    'max_depth': hp.quniform('max_depth', 2, 20, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'n_estimators': hp.uniform('n_estimators', 100, 1000),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'reg_alpha': hp.uniform('reg_alpha', 0, 0.1),
    'reg_lamda': hp.uniform('reg_lambda', 0, 10),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 0, 1)
}

trials, best = optimize_model_stratifiedkfold(target_column='cancer_specific_mortality',
                                              max_time=216,
                                              space=space_model,
                                              df = data_train.copy(),
                                              model_type='XGBoost',
                                              max_evals=80)

#### death from other causes

In [ ]:
space_model = {
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'gamma': hp.quniform('gamma', 0, 1, 0.05),
    'learning_rate': hp.quniform('learning_rate', 0, 0.2, 0.025),
    'max_depth': hp.quniform('max_depth', 2, 20, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'n_estimators': hp.uniform('n_estimators', 100, 1000),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'reg_alpha': hp.uniform('reg_alpha', 0, 0.1),
    'reg_lamda': hp.uniform('reg_lambda', 0, 10),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 0, 1)
}

trials, best = optimize_model_stratifiedkfold(target_column='death_from_other_causes',
                                              max_time=216,
                                              space=space_model,
                                              df = data_train.copy(),
                                              model_type='XGBoost',
                                              max_evals=80)

#### mts

In [ ]:
space_model = {
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'gamma': hp.quniform('gamma', 0, 1, 0.05),
    'learning_rate': hp.quniform('learning_rate', 0, 0.2, 0.025),
    'max_depth': hp.quniform('max_depth', 2, 20, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'n_estimators': hp.uniform('n_estimators', 100, 1000),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'reg_alpha': hp.uniform('reg_alpha', 0, 0.1),
    'reg_lamda': hp.uniform('reg_lambda', 0, 10),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 0, 1)
}

trials, best = optimize_model_stratifiedkfold(target_column='mts',
                                              max_time=216,
                                              space=space_model,
                                              df = data_train.copy(),
                                              model_type='XGBoost',
                                              max_evals=80)

#### bcr

In [ ]:
space_model = {
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'gamma': hp.quniform('gamma', 0, 1, 0.05),
    'learning_rate': hp.quniform('learning_rate', 0, 0.2, 0.025),
    'max_depth': hp.quniform('max_depth', 2, 20, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'n_estimators': hp.uniform('n_estimators', 100, 1000),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'reg_alpha': hp.uniform('reg_alpha', 0, 0.1),
    'reg_lamda': hp.uniform('reg_lambda', 0, 10),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 0, 1)
}

trials, best = optimize_model_stratifiedkfold(target_column='bcr',
                                              max_time=216,
                                              space=space_model,
                                              df = data_train.copy(),
                                              model_type='XGBoost',
                                              max_evals=80)

### Artificial neural network

#### cancer specific mortality

In [11]:
space_model = {'layer_size':hp.quniform('layer_size', 25, 100, 1),
         'alpha':hp.lognormal('alpha', mu=np.log(1e-4), sigma=1),
         'activation':hp.choice('activation', ['logistic', 'tanh', 'relu']),
         'learning_rate':hp.loguniform('learning_rate', low=np.log(1e-4), high=np.log(1.)),
        }

trials, best = optimize_model_stratifiedkfold(target_column='cancer_specific_mortality',
                                              max_time=216,
                                              space=space_model,
                                              df = data_train.copy(),
                                              model_type='NN',
                                              max_evals=80)

100%|██████████| 80/80 [51:17<00:00, 38.47s/trial, best loss: -0.8689690596625324] 
{'activation': 0, 'alpha': 9.190404654691428e-05, 'layer_size': 73.0, 'learning_rate': 0.0017325216661984056}


In [25]:
space_model = {'hidden_layer_sizes':hp.choice('hidden_layer_sizes', [8, 16, 32, (8,8), (16,16)]), # 8, 16, 32, (8,8), (16,16)
         'alpha':hp.lognormal('alpha', mu=np.log(1e-4), sigma=1),
         'activation':hp.choice('activation', ['logistic', 'tanh', 'relu']),
         'learning_rate':hp.choice('learning_rate', ['constant','adaptive']),
         'max_iter': hp.choice('max_iter', [500]),
         'solver': hp.choice('solver', ['sgd', 'adam', 'lbfgs']),
        }

trials, best = optimize_model_stratifiedkfold(target_column='cancer_specific_mortality',
                                              max_time=216,
                                              space=space_model,
                                              df = data_train.copy(),
                                              model_type='NN',
                                              max_evals=80)

 11%|█▏        | 9/80 [02:41<21:43, 18.36s/trial, best loss: -0.8183074098461336] 

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: C

 12%|█▎        | 10/80 [02:56<19:57, 17.11s/trial, best loss: -0.8183074098461336]

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)



 44%|████▍     | 35/80 [20:48<1:08:38, 91.51s/trial, best loss: -0.8718663275528582]

...

 89%|████████▉ | 71/80 [1:06:30<25:18, 168.73s/trial, best loss: -0.8929078928729268]

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)



 98%|█████████▊| 78/80 [1:15:15<01:35, 47.99s/trial, best loss: -0.8929078928729268] 

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)



100%|██████████| 80/80 [1:19:31<00:00, 59.65s/trial, best loss: -0.8929078928729268]
{'activation': 2, 'alpha': 3.670399366093249e-06, 'hidden_layer_sizes': 4, 'learning_rate': 1, 'max_iter': 0, 'solver': 2}


In [26]:
space_model = {'hidden_layer_sizes':hp.choice('hidden_layer_sizes', [8, 16, 32, (8,8), (16,16), (32, 16), (32, 32)]), # 8, 16, 32, (8,8), (16,16)
         'alpha':hp.lognormal('alpha', mu=np.log(1e-4), sigma=1),
         'activation':hp.choice('activation', ['logistic', 'tanh', 'relu']),
         'learning_rate':hp.choice('learning_rate', ['constant','adaptive']),
         'max_iter': hp.choice('max_iter', [2000]),
         'solver': hp.choice('solver', ['sgd', 'adam', 'lbfgs']),
        }

trials, best = optimize_model_stratifiedkfold(target_column='cancer_specific_mortality',
                                              max_time=216,
                                              space=space_model,
                                              df = data_train.copy(),
                                              model_type='NN',
                                              max_evals=80)

 18%|█▊        | 14/80 [21:59<1:22:30, 75.01s/trial, best loss: -0.8755981332541104] 

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: C

 19%|█▉        | 15/80 [22:15<1:01:49, 57.06s/trial, best loss: -0.8755981332541104]

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)



 68%|██████▊   | 54/80 [47:40<21:54, 50.56s/trial, best loss: -0.8755981332541104]  

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: C

 69%|██████▉   | 55/80 [47:55<16:42, 40.10s/trial, best loss: -0.8755981332541104]

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)



100%|██████████| 80/80 [58:04<00:00, 43.56s/trial, best loss: -0.8755981332541104]
{'activation': 1, 'alpha': 0.0002340750505402387, 'hidden_layer_sizes': 2, 'learning_rate': 0, 'max_iter': 0, 'solver': 1}


In [27]:
space_model = {'hidden_layer_sizes':hp.choice('hidden_layer_sizes', [8, 16, 32, 64, 128, 
                                                                     (8,8), (16,16), (32, 16), (32, 32),
                                                                     (16, 8), (128, 64, 32, 16), (32, 16, 16)]), # 8, 16, 32, (8,8), (16,16)
         'alpha':hp.lognormal('alpha', mu=np.log(1e-4), sigma=1),
         'activation':hp.choice('activation', ['logistic', 'tanh', 'relu']),
         'learning_rate':hp.choice('learning_rate', ['constant','adaptive']),
         'max_iter': hp.choice('max_iter', [2500]),
         'solver': hp.choice('solver', ['sgd', 'adam', 'lbfgs']),
        }

trials, best = optimize_model_stratifiedkfold(target_column='cancer_specific_mortality',
                                              max_time=216,
                                              space=space_model,
                                              df = data_train.copy(),
                                              model_type='NN',
                                              max_evals=80)

  9%|▉         | 7/80 [14:01<1:36:48, 79.56s/trial, best loss: -0.8440756313487654] 

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: C

 10%|█         | 8/80 [14:34<1:17:58, 64.97s/trial, best loss: -0.8440756313487654]

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)



 96%|█████████▋| 77/80 [1:17:21<02:25, 48.42s/trial, best loss: -0.8678197664611105] 

c:\Users\Vytautas\Studijos\prostate-cancer-mortality\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)



100%|██████████| 80/80 [1:29:17<00:00, 66.96s/trial, best loss: -0.8678197664611105] 
{'activation': 1, 'alpha': 6.835502772513517e-05, 'hidden_layer_sizes': 11, 'learning_rate': 0, 'max_iter': 0, 'solver': 1}


#### death from other causes

In [12]:
space_model = {'layer_size':hp.quniform('layer_size', 25, 100, 1),
         'alpha':hp.lognormal('alpha', mu=np.log(1e-4), sigma=1),
         'activation':hp.choice('activation', ['logistic', 'tanh', 'relu']),
         'learning_rate':hp.loguniform('learning_rate', low=np.log(1e-4), high=np.log(1.)),
         'max_iter': hp.choice('max_iter', [1000]),
        }

trials, best = optimize_model_stratifiedkfold(target_column='death_from_other_causes',
                                              max_time=216,
                                              space=space_model,
                                              df = data_train.copy(),
                                              model_type='NN',
                                              max_evals=80)

100%|██████████| 80/80 [47:16<00:00, 35.46s/trial, best loss: -0.5927730456162876]
{'activation': 2, 'alpha': 0.0002616927777929669, 'layer_size': 70.0, 'learning_rate': 0.0008885558916449781}


#### bcr

In [13]:
space_model = {'layer_size':hp.quniform('layer_size', 25, 100, 1),
         'alpha':hp.lognormal('alpha', mu=np.log(1e-4), sigma=1),
         'activation':hp.choice('activation', ['logistic', 'tanh', 'relu']),
         'learning_rate':hp.loguniform('learning_rate', low=np.log(1e-4), high=np.log(1.)),
        }

trials, best = optimize_model_stratifiedkfold(target_column='bcr',
                                              max_time=216,
                                              space=space_model,
                                              df = data_train.copy(),
                                              model_type='NN',
                                              max_evals=80)

100%|██████████| 80/80 [42:38<00:00, 31.98s/trial, best loss: -0.847685297272505] 
{'activation': 0, 'alpha': 0.005821356388227255, 'layer_size': 34.0, 'learning_rate': 0.0002702352915791481}


#### mts

In [10]:
space_model = {'layer_size':hp.quniform('layer_size', 25, 100, 1),
         'alpha':hp.lognormal('alpha', mu=np.log(1e-4), sigma=1),
         'activation':hp.choice('activation', ['logistic', 'tanh', 'relu']),
         'learning_rate':hp.loguniform('learning_rate', low=np.log(1e-4), high=np.log(1.)),
        }

trials, best = optimize_model_stratifiedkfold(target_column='mts',
                                              max_time=216,
                                              space=space_model,
                                              df = data_train.copy(),
                                              model_type='NN',
                                              max_evals=80)

100%|██████████| 80/80 [32:44<00:00, 24.56s/trial, best loss: -0.8832526100502556] 
{'activation': 2, 'alpha': 1.0637827193154231e-05, 'layer_size': 45.0, 'learning_rate': 0.025594709065934664}
